In [ ]:
from pathlib import Path

base_path = Path(
    r"C:/Users/saraj/OneDrive/Escritorio/Past semesters/10. Fall 24 (Senior)/CSE 570/Project/Jumps"
)

person_to_file = {
    "kevin": base_path / "Kevin_Wei.pkl",
    "alireza": base_path / "Alireza_Aghaei.pkl",
    "tanmay": base_path / "Tanmay.pkl",
    "michael": base_path / "Michael.pkl",
}

In [22]:
ground_truth_values = {
    "michael": {
        1: {
            "takeoff_knee_bend": 42.9,
            "landing_knee_bend": 42.0,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        2: {
            "takeoff_knee_bend": 45.3,
            "landing_knee_bend": 45.8,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        3: {
            "takeoff_knee_bend": 41.4,
            "landing_knee_bend": 50.6,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        4: {
            "takeoff_knee_bend": 38.6,
            "landing_knee_bend": 39.8,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        5: {
            "takeoff_knee_bend": 78.5,
            "landing_knee_bend": 52.1,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        6: {
            "takeoff_knee_bend": 58.3,
            "landing_knee_bend": 57.3,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        7: {
            "takeoff_knee_bend": 68.0,
            "landing_knee_bend": 94.1,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        8: {
            "takeoff_knee_bend": 47.0,
            "landing_knee_bend": 55.5,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
    },
    "kevin": {
        1: {
            "takeoff_knee_bend": 41.2,
            "landing_knee_bend": 32.6,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        2: {
            "takeoff_knee_bend": 57.8,
            "landing_knee_bend": 35.7,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        3: {
            "takeoff_knee_bend": 102.6,
            "landing_knee_bend": 34.9,
            "arm_swing_magnitude": 80.1,
            "height": 0,
        },
        4: {
            "takeoff_knee_bend": 40.0,
            "landing_knee_bend": 37.8,
            "arm_swing_magnitude": 164.3,
            "height": 0,
        },
        5: {
            "takeoff_knee_bend": 48.6,
            "landing_knee_bend": 57.4,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        6: {
            "takeoff_knee_bend": 34.8,
            "landing_knee_bend": 24.5,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
    },
    "tanmay": {
        1: {
            "takeoff_knee_bend": 49.7,
            "landing_knee_bend": 24.3,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        2: {
            "takeoff_knee_bend": 42.6,
            "landing_knee_bend": 24.9,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        3: {
            "takeoff_knee_bend": 80.9,
            "landing_knee_bend": 22.2,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        4: {
            "takeoff_knee_bend": 49.8,
            "landing_knee_bend": 32.9,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        5: {
            "takeoff_knee_bend": 49.3,
            "landing_knee_bend": 27.8,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
    },
    "alireza": {
        1: {
            "takeoff_knee_bend": 48.6,
            "landing_knee_bend": 34.3,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        2: {
            "takeoff_knee_bend": 88.0,
            "landing_knee_bend": 27.3,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        3: {
            "takeoff_knee_bend": 45.9,
            "landing_knee_bend": 58.3,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        4: {
            "takeoff_knee_bend": 54.1,
            "landing_knee_bend": 24.8,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
        5: {
            "takeoff_knee_bend": 53.3,
            "landing_knee_bend": 27.2,
            "arm_swing_magnitude": 0,
            "height": 0,
        },
    },
}

In [28]:
import pickle
from jump_detection import Jump


def load_jump_summaries(pickle_path, person_id, ground_truth):
    with open(pickle_path, "rb") as f:
        raw_jump_list = pickle.load(f)

    results = []

    for i, old_jump in enumerate(raw_jump_list, start=1):
        try:
            # Recreate jump to trigger fresh metric calculation
            jump = Jump(
                lower_back_accel=old_jump.lower_back_accel,
                lower_back_gyro=old_jump.lower_back_gyro,
                wrist_accel=old_jump.wrist_accel,
                wrist_gyro=old_jump.wrist_gyro,
                thigh_accel=old_jump.thigh_accel,
                thigh_gyro=old_jump.thigh_gyro,
                detected_time=old_jump.detected_time,
            )

            if jump.metrics is None:
                continue

            gt = ground_truth.get(person_id, {}).get(i)
            if gt is None:
                continue  # No ground truth to compare for this jump

            comparison = {
                "person_id": person_id,
                "jump_id": i,
            }

            for metric in gt:
                pred = jump.metrics.get(metric)
                truth = gt[metric]
                comparison[f"predicted_{metric}"] = pred
                comparison[f"gt_{metric}"] = truth

                if pred is not None and truth is not None:
                    comparison[f"{metric}_error"] = abs(pred - truth)
                else:
                    comparison[f"{metric}_error"] = None


            results.append(comparison)

        except Exception as e:
            print(f"Error processing {person_id} jump {i}: {e}")

    return results

In [34]:
import pandas as pd

all_results = []



for person, file_path in person_to_file.items():
    results = load_jump_summaries(file_path, person, ground_truth_values)
    all_results.extend(results)


metrics = [key.replace("gt_", "") for key in all_results[0] if key.startswith("gt_")]

# Print one table per metric
for metric in metrics:
    if metric == "arm_swing_magnitude":
        print(f"\nMetric: {metric}")
        print(f"{'Person':<14} {'Jump':<6} {'Ground Truth':<14} {'Predicted':<14} {'Error':<14}")
        print("-" * 62)

        errors = []
        for row in all_results:
            gt = row.get(f"gt_{metric}")
            pred = row.get(f"predicted_{metric}")
            err = row.get(f"{metric}_error")

            if gt is not None and pred is not None and err is not None:
                errors.append(err)
                print(
                    f"{row['person_id']:<14} {row['jump_id']:<6} {gt:<14.3f} {pred:<14.3f} {err:<14.3f}"
                )

        if errors:
            avg_error = sum(errors) / len(errors)
            print("-" * 62)
            print(f"{'Average Error':<34} {avg_error:<14.3f}")


Metric: arm_swing_magnitude
Person         Jump   Ground Truth   Predicted      Error         
--------------------------------------------------------------
